In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = ''
import tensorflow as tf
tf.enable_eager_execution()
import numpy as np

In [2]:
raw_labels = [
    ['./data/image/train/7f239a488dad0640.jpg', [553, 440, 170, 376, 348, 281, 511, 413, 415, 210, 77, 284, 242], []],
    ['./data/image/train/c05aa68eef6bbb19.jpg', [156, 347, 138, 195, 413, 290, 210, 549, 535, 284, 121], [493, 415, 534, 535, 121, 468]],
    ['./data/image/train/2ad3d44a2cfc17d8.jpg', [53, 441], []],
    ['./data/image/train/48e5fab41cd8de5e.jpg', [156, 138, 195, 181, 222, 415, 210, 77, 284], [222, 181]],
    ['./data/image/train/0a4f0fddae6809ac.jpg', [284], []],
    ['./data/image/train/5bd244ec3611291b.jpg', [441, 527, 485, 207, 27], []],
    ['./data/image/train/f9b1fd9e8dadf499.jpg', [221, 13, 413], [486, 499, 221, 13]],
    ['./data/image/train/6a93893fc654b6f7.jpg', [531, 77, 497, 413], [219, 291, 7, 440]],
    ['./data/image/train/17a1fa0e433cc6cb.jpg', [493, 156, 409, 138, 195, 415, 210, 284], [16]],
    ['./data/image/train/f8b9c0a8df55e958.jpg', [441, 527, 485, 207, 27], [138]],
    ['./data/image/train/5349ae94e7b75871.jpg', [347, 156, 38, 501, 413, 290, 210, 535, 273, 284, 121, 494], [199, 126, 501, 38, 494]],
    ['./data/image/train/33b1a64ab73882ba.jpg', [156, 342, 210], []],
    ['./data/image/train/fd83c801d8fed523.jpg', [499, 13], [37, 374, 566, 448, 149, 375, 568, 502, 76, 43, 299, 342, 380, 16, 200, 156, 384, 83, 386, 457, 423, 88, 233, 541, 391, 309, 162, 165, 91, 245, 24, 394, 246, 25, 548, 518, 254, 210, 212, 551, 286, 287, 524, 181, 439, 187, 66, 29, 104, 595, 216, 192, 444, 262, 598, 109, 36]],
    ['./data/image/train/03d9633a53f4df39.jpg', [170, 528, 413, 251, 112], [253, 2, 235, 323, 313]],
    ['./data/image/train/4e56878699946995.jpg', [527, 485, 400, 441, 207], [486, 499, 221, 13]],
    ['./data/image/train/28ae567a4b0f8baf.jpg', [284], [534, 121, 535, 468]],
    ['./data/image/train/7700897d266857a3.jpg', [284, 192, 210, 415], [221, 499, 40, 161, 21, 30, 243, 190, 486, 340, 13]],
    ['./data/image/train/67a4c9e6117f3947.jpg', [493, 499, 415, 210, 284, 13, 16], [486, 499, 221, 13]],
    ['./data/image/train/8d3a867fa16e46bc.jpg', [534, 121], [203, 600, 594, 139]],
    ['./data/image/train/a6a74360298775c6.jpg', [582, 221, 13, 53, 441], [221, 574, 441, 499, 231, 485, 53, 43, 486, 488, 13]],
    ['./data/image/train/2c41215cffd76a5e.jpg', [38, 501, 415, 210, 535, 340, 284, 121, 494], []],
    ['./data/image/train/0e277c67b1e0c7a8.jpg', [409, 138, 195, 210, 162, 489, 212, 284], [493, 549]],
]

From raw labels to tensorflow labels

In [17]:
labels_np = np.zeros((len(raw_labels), 601), dtype=np.int32)
weights_np = np.zeros((len(raw_labels), 601), dtype=np.int32)

In [18]:
for ii, l in enumerate(raw_labels):
    labels_np[ii, l[1]] = 1
    if not l[2]:
        weights_np[ii, :] = 1
    else:
        weights_np[ii, l[1] + l[2]] = 1

In [26]:
labels = tf.constant(labels_np)
weights = tf.constant(weights_np)

Test loss computation with random logits

In [27]:
logits_np = np.random.uniform(-5, 5, labels_np.shape)

In [30]:
logits = tf.constant(logits_np, dtype=tf.float32)

In [33]:
sigmoid_loss = tf.nn.sigmoid_cross_entropy_with_logits(labels=tf.to_float(labels), logits=logits, name='sigmoid_ce')
sigmoid_loss = tf.reduce_mean(sigmoid_loss * tf.to_float(weights), name='sigmoid_loss')

In [53]:
softmax = tf.nn.softmax(logits, axis=-1)
# reweight softmax
w_softmax = softmax * tf.to_float(weights)
w_softmax = tf.truediv(w_softmax, tf.reduce_sum(w_softmax, axis=-1, keepdims=True))
# loss = -log(sum of probs from all positive labels)
sum_w_softmax = tf.reduce_sum(w_softmax * tf.to_float(labels), axis=-1, keepdims=True)
softmax_loss = tf.reduce_mean(-tf.log(sum_w_softmax), name='softmax_loss')

In [70]:
_, topk = tf.nn.top_k(logits, k=1)

TypeError: tuple indices must be integers or slices, not tuple

In [74]:
best1_acc = tf.reduce_max(tf.one_hot(topk[:, 0], depth=601) * tf.to_float(labels), axis=1)

In [75]:
tf.reduce_mean(best1_acc)

<tf.Tensor: id=209, shape=(), dtype=float32, numpy=0.0>

In [1]:
def bilinear(values, xy_w):
    '''
    p00 p01
    p10 p11
    '''
    p00, p01, p10, p11 = values
    wx, wy = xy_w
    y0 = p00 * wx + p01 * (1.0 - wx)
    y1 = p10 * wx + p11 * (1.0 - wx)
    return y0 * wy + y1 * (1.0 - wy)

In [13]:
bilinear([1, 2, 6, 7], [0.75, 0.25])

5.0